# 1. PyPicoScenes
PyPicoScenes是基于C++Wi-Fi通感一体化（ISAC）研究框架[PicoScenes](https://ps.zpj.io/index.html)的Python绑定库，通过Cppyy的动态绑定技术，实现了底层C++ API的无缝封装，为科研人员提供兼具高性能与开发效率的Python编程接口。Python版本完整继承了原平台的硬件兼容性与算法创新性，同时深度融入Python生态，显著降低了Wi-Fi感知与通信协同研究的开发门槛。  

# 2. 安装
PyPicoScenes的运行依赖于python的cppyy库以及PicoScenes的头文件和动态库。当前PyPicoScenes支持Ubuntu平台和Mac平台，以Ubutnu环境为例，其安装遵循如下几个步骤：
1. PicoScenes安装  
[PicoScenes的安装请参考这里](https://ps.zpj.io/installation.html#picoscenes-software-installation)。
2. Anaconda安装  
[Anaconda的安装请参考这里](https://anaconda.org/anaconda/conda)。
3. 安装cppyy  
cppyy 是一个基于 ​Cling/LLVM 的动态绑定工具，通过运行时解析 C++ 代码实现与 Python 的无缝交互。其核心优势在于无需预编译绑定代码，支持从 C++98 到 C++20 的多种标准，并兼容 PyPy 和 CPython 解释器。我们推荐使用[Anaconda](https://www.anaconda.com/download/success)环境安装cppyy其其它依赖项。
    * conda create -n ENV_NAME python=3.10
    * conda activate ENV_NAME
    * pip install -r requirements

# 3. Cppyy封装PicoScenens
[Cppyy](https://cppyy.readthedocs.io/en/latest/#) 是一款基于 ​Cling 解释器的动态运行时 Python-C++ 双向绑定工具，通过即时解析 C++ 代码生成高效接口，实现两种语言的深度互操作。其核心价值在于零手工封装、高性能、低内存开销，支持复杂场景如跨语言继承、模板实例化及异常映射，极大简化了 Python 调用 C++ 库的流程，尤其适合大规模项目与交互式开发。下面的例子说明如何使用cppyy调用C++的STL库，实现python与C++的交互。
```python
    # 使用 C++ vector
    import cppyy
    cppyy.include("vector")
    # C++符号都在cppyy.gbl中，通过cppyy.gbl使用C++中的符号
    vec = cppyy.gbl.std.vector[int](3,1)
    print(vec) # 输出{ 1, 1, 1 }
    # 在python中使用vector<int>的size()方法
    print(vec.size())
    vec.push_back(5)
    print(vec) # 输出{ 1, 1, 1, 5 }
    print(vec.size())
```
接下来说明如何使用cppyy封装picoscenes。
## 3.1. 将PicoScenes头文件加入cppyy_include_path  
假设PicoScenes的绝对安装路径为**your_picoscenes_path**，首先将头文件以及动态库加入cppyy路径：
```python
    import cppyy
    import cppyy.ll
    # 添加头文件路径
    cppyy.add_include_path("your_picoscenes_path/include")
    # 添加动态库路径在导入完头文件以及动态库之后，C++中所有符号都在cppyy.gbl命名空间中，可以通过cppyy.gbl在python中使用C++
## 3.2. 导入C++头文件
使用cppyy.include导入所需头文件：
```python
    cppyy.include("PicoScenes/SystemTools.hxx")
    cppyy.include("PicoScenes/QCA9300FrontEnd.hxx")
    cppyy.include("PicoScenes/IntelRateNFlag.hxx")
    cppyy.include("PicoScenes/AbstractSDRFrontEnd.hxx")
    cppyy.include("PicoScenes/USRPFrontEnd.hxx")
    # 以及其它所需的头文件
```
## 3.3. 导入动态库
使用cppyy.load_library导入所需动态库：
```python
    cppyy.load_library("libDSP")
    cppyy.load_library("libFrontEnd")
    cppyy.load_library("libIntrinsics")
    cppyy.load_library("libLicense")
    cppyy.load_library("libmac80211Injection")
    cppyy.load_library("libNICHAL")
    cppyy.load_library("librxs_parsing")
    # 以及其它所需的动态库
```
## 3.4. 使用C++ API
在导入完头文件以及动态库之后，C++中所有符号都在cppyy.gbl命名空间中，可以通过cppyy.gbl在python中使用C++。例如要获取usrp：
```python
    nicName = "usrp"
    nic = cppyy.gbl.NICPortal.getInstance().getNIC(nicName)
    print(nic)
    # nic is a AbstractNIC object such as:
    # nic: <cppyy.gbl.AbstractNIC object at 0x561264e43b28 held by std::shared_ptr<AbstractNIC> at 0x561264e25c40>

    # start nic's RxService
    nic.startRxService()
    # stop nic's RxService
    nic.stopRxService()
```

# 3. 快速开始
PyPicoScenes封装了PicoScenes底层的核心API，使得开发者无需实现复杂的[C++插件](https://github.com/wifisensing/PicoScenes-PDK)，通过Python接口即可实现WiFi包的收发、CSI测量等功能。当需要使用网卡或者usrp的收发功能时，请按照以下流程操作：  
1. 平台初始化  
执行picoscenes_start()启动PicoScenes运行时环境。
2. 硬件获取  
通过getNic获取指定类型硬件设备。
3. 硬件参数配置  
使用python api设置硬件前端射频参数（采样率、带宽、中心频率等）。
4. 发送参数设置(如果需要发送WiFi包)  
使用python api设置网卡的发包参数（WiFi帧格式、MCS、STS等）。
5. 具体功能实现  
启动网卡的发送与接收服务，使用对应硬件的底层api执行数据发送/接收操作。
6. 注册python回调  
PyPicoScenes支持通过注册Python回调函数来处理接收到的WiFi数据包。需要特别注意的是，注册的回调函数的第一个形式参数必须表示WiFi数据包，而后续的所有形式参数都必须提供默认值。例如：
```python
    def call_back(frame, arg1=1, arg2=2, arg3=3,...)
```
7. 运行控制  
调用picoscenes_wait阻塞主线程以维持平台运行。
8. 平台终止  
关闭网卡的发送与接收服务，并执行picoscenes_stop停止平台（在此之前，picoscenes_wait将保持阻塞状态）。

接下来从如下5个方面来说明如何使用PyPicoScenes：
1. CSI文件解析。
2. 使用SDR设备接收WiFi包并测量CSI。
3. 使用SDR设备发送WiFi包。
4. 使用AX210/AX200网卡接收WiFi包并测量CSI。
5. 使用AX210/AX200网卡发送WiFi包。

## 3.1. CSI文件解析
二进制CSI文件**rx_by_usrpN210.csi**由PicoScenes通过USRP N210设备采集生成，可通过调用ModularPicoScenesRxFrame类的fromBuffer类方法直接从原始字节缓冲区反序列化解码出完整的帧对象。

In [ ]:
import struct
import numpy as np
from PyPicoScenes import *
import os

# parseCSIFile会解析出filename文件中从pos位置开始的num个frame
def parseCSIFile(filename: str = "", pos: int = 0, num: int = 0):
    res = []
    count = 0
    try:
        with open(filename, "rb") as f:
            # 获取文件总长度
            f.seek(0, os.SEEK_END)
            lens = f.tell()
            f.seek(0, os.SEEK_SET)
            if num == 0:
                num = lens - pos  # 读取剩余全部内容
            while pos < (lens - 4) and count < num:
                header = f.read(4)
                if len(header) < 4:
                    break
                # 解析字段长度（需根据实际字节序调整）
                field_len = struct.unpack("<I", header)[0] + 4 # 假设小端序
                # 回退指针并读取完整字段
                f.seek(-4, os.SEEK_CUR)

                data = f.read(field_len)
                if len(data) < field_len:
                    break
                
                frame = ModularPicoScenesRxFrame.fromBuffer(np.frombuffer(data, dtype=np.uint8), field_len, True)
                if frame:
                    res.append(frame)
                pos += field_len
                count += 1

    except Exception as e:
        print(f"Failed to parse csi file.: {e}")
        return []
    return res

fileName = "rx_by_usrpN210.csi"
res = parseCSIFile(fileName)
print(res[0])

## 3.2. 使用SDR设备接收WiFi包并测量CSI
本模块详解如何通过PyPicoScenes API驱动SDR设备实现Wi-Fi协议栈物理层帧接收。PyPicoScenes支持通过精细化API控制SDR设备（如USRP）实现802.11a/g/n/ac/ax/be全协议Wi-Fi帧接收及信道状态信息（CSI）测量，开发者可通过以下核心参数动态配置接收链路：​​多通道选择​​（setRxChannels指定物理通道索引）、​​射频带宽（setRxChannelBandwidthMode设置接收包带宽，需通过setRxSamplingRate设置匹配的采样率）、​​中心频率​​（setCarrierFrequency定义2.4/5/6GHz频段）、​​接收增益​​（setRxGain支持相对比例或绝对功率模式）、​​自动增益控制​​（setAGC启用/禁用射频前端动态调节）、​​天线阵列​​（setRxAntennas指定"TX/RX"或"RX2"等硬件端口）、​​信号重采样率​​（setRxResampleRatio优化基带处理）以及​​多线程解码​​（setNumThreads4RxDecoding提升实时性），同时支持注册Python回调函数（registerGeneralHandler）。

### 3.2.1. 单usrp单通道WiFi包接收

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess

FrameDumper = cppyy.gbl.FrameDumper

# 简单的回调函数
def py_call_back(frame):
    print("-----------------------------get one frame----------------------------")
    return True

# python回调接收到frame并将frame保存到文件
def py_call_back_dump(frame):
    fileName = "testCSI"
    print(f"dump a frame to {fileName}")
    # 保存frame到文件
    FrameDumper.getInstanceWithoutTime(fileName).dumpRxFrame(frame)
    return True

def recv_frame(nicName:str = 'usrp'):
    # 以root权限执行当前脚本
    run_as_root()
    # result = subprocess.run(["PicoScenes", "-q"], capture_output=True, text=True)
    result = subprocess.run(["array_prepare_for_picoscenes", nicName ,"5200 HT20"], capture_output=True, text=True)
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取usrp或者网卡设备
    nic = getNic(nicName)
    # 获取相应的硬件前端
    frontEnd = nic.getTypedFrontEnd[AbstractSDRFrontEnd]()

    # 参数设置
    # 设置rxcm为1（rx-channel:[0]）
    rxChannelList = [0]
    frontEnd.setRxChannels(rxChannelList)
    
    ## 设置接收包格式为RX_CBW_20
    ### 设置接收采样率为20MHz
    if (not frontEnd.getHardwareSupportedRxChannels().empty()):
        frontEnd.setRxSamplingRate(20e6)
    ### 设置rx-resample-ratio
    frontEnd.setRxResampleRatio(1.0)
    ### 设置rx-cbw: 20
    frontEnd.setRxChannelBandwidthMode(ChannelBandwidthEnum(20))
    
    '''
    ## 设置接收包格式为RX_CBW_40
    ### 设置接收采样率为40MHz
    if (not frontEnd.getHardwareSupportedRxChannels().empty()):
        frontEnd.setRxSamplingRate(40e6)
    ### 设置rx-resample-ratio
    frontEnd.setRxResampleRatio(1.0)
    ### 设置rx-cbw: 40
    frontEnd.setRxChannelBandwidthMode(ChannelBandwidthEnum(40))
    '''
    
    '''
    ## 设置接收包格式为RX_CBW_80
    ### 设置接收采样率为100MHz
    if (not frontEnd.getHardwareSupportedRxChannels().empty()):
        frontEnd.setRxSamplingRate(100e6)
    ### 设置rx-resample-ratio
    frontEnd.setRxResampleRatio(1.0)
    ### 设置rx-cbw: 80
    frontEnd.setRxChannelBandwidthMode(ChannelBandwidthEnum(80))
    '''
    
    # 设置时钟源，默认为'internal'
    frontEnd.setClockSource("internal")
    frontEnd.setTimeSource("internal")
    
    # 设置中心频率为2412MHz
    frontEnd.setCarrierFrequency(2412e6)
    
    # 设置接收功率为最大接收功率的0.65
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setRxGain(0.65)
    '''
    # 设置接收功率为20dBm
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setRxGain(20)
    '''
    
    # 设置agc，默认关闭
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setAGC(False)
        
    # 设置接收天线，默认为"TX/RX"
    frontEnd.setRxAntennas(["TX/RX", "TX/RX"])
    '''
    # 设置接收天线为"RX2"
    frontEnd.setRxAntennas(["RX2"])
    '''
    
    # 设置N-thread multithread Rx decoding, 默认为1
    frontEnd.setNumThreads4RxDecoding(1)
    
    # 启动网卡接收服务
    nic.startRxService()
    
    # 注册python回调
    call_backs = {
        "call_back" : py_call_back,
        "call_back_dump" : py_call_back_dump,
    }
    for call_back_name, call_back in call_backs.items():
        nic.registerGeneralHandler(call_back_name, call_back)
        
    while (True):
        pass
    
    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

recv_frame()

`rx-channel`和`rxcm`是功能完全相同的两个选项，均用于配置接收通道，只是参数格式不同：`rx-channel`采用直接列举通道编号的方式（如`0,1,2,3`），而`rxcm`使用位掩码形式（如`15`对应二进制`1111`表示启用射频通道[0,1,2,3]）。需要特别注意的是，多通道接收仅代表物理层资源的配置，并不等同于MIMO传输技术。`rxcm`编码规则如下：
* 1（二进制为01）：仅第1接收通道，对应channelList为[0]；
* 2（二进制为10）：仅第2接收通道，对应channelList为[1]；
* 3（二进制为11）：第1和第2通道，对应channelList为[0,1]；
* 4（二进制100）：第3接收通道，对应channelList为[2]；

### 3.2.2. 单usrp多通道WiFi包接收
<!-- PyPicoScenes支持多通道接收与多USRP联合多通道接收。例如，NI USRP B210、X310等高端型号具备两个及以上独立射频通道。PyPicoScenes可实现双通道/多通道信号接收及MIMO帧解码。执行下面这段代码，可以在X310等具备多信道能力的USRP设备，在中心频率5190MHz（对应5180 HT40+或5200 HT40-频道）的40MHz带宽上，通过双接收通道（通过**setRxChannels** api设置接收通道）监听Wi-Fi流量。   -->
PicoScenes支持单USRP设备多通道接收及多USRP联合组网接收模式。以NI USRP B210/X310等高端型号为例，该类设备配备两个及以上独立射频通道，可通过PicoScenes实现双通道/多通道信号同步接收，并支持MIMO（多入多出）帧解码功能。执行下面代码，可在X310等多通道USRP设备上监听中心频率5190MHz（对应5180MHz HT40+或5200MHz HT40-信道）的40MHz带宽Wi-Fi空口流量，并通过双接收通道执行信号捕获。


In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess

FrameDumper = cppyy.gbl.FrameDumper

# 简单的回调函数
def py_call_back(frame):
    print("-----------------------------get one frame----------------------------")
    return True

# python回调接收到frame并将frame保存到文件
def py_call_back_dump(frame):
    fileName = "testCSI"
    print(f"dump a frame to {fileName}")
    # 保存frame到文件
    FrameDumper.getInstanceWithoutTime(fileName).dumpRxFrame(frame)
    return True

def recv_frame(nicName:str = 'usrp'):
    # result = subprocess.run(["PicoScenes", "-q"], capture_output=True, text=True)
    # 以root权限执行当前脚本
    run_as_root()
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取usrp或者网卡设备
    nic = getNic(nicName)
    # 获取相应的硬件前端
    frontEnd = nic.getTypedFrontEnd[AbstractSDRFrontEnd]()

    # 参数设置
    # 设置rxcm为3（rx-channel:[0,1]）
    rxChannelList = [0,1]
    frontEnd.setRxChannels(rxChannelList)
    
    ## 设置接收包格式为RX_CBW_40
    if (not frontEnd.getHardwareSupportedRxChannels().empty()):
        frontEnd.setRxSamplingRate(40e6)
    ### 设置rx-resample-ratio
    frontEnd.setRxResampleRatio(1.0)
    ### 设置rx-cbw: 40
    frontEnd.setRxChannelBandwidthMode(ChannelBandwidthEnum(40))
    
    # 设置时钟源，默认为'internal'
    frontEnd.setClockSource("internal")
    frontEnd.setTimeSource("internal")
    
    # 设置中心频率为5190MHz
    frontEnd.setCarrierFrequency(5190e6)
    
    # 设置接收功率为最大接收功率的0.65
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setRxGain(0.65)
    
    # 设置agc，默认关闭
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setAGC(False)
        
    # 设置接收天线，默认为"TX/RX"
    frontEnd.setRxAntennas(["TX/RX", "TX/RX"])
    
    # 设置N-thread multithread Rx decoding, 默认为1
    # frontEnd.setNumThreads4RxDecoding(1)
    
    # 启动网卡接收服务
    nic.startRxService()
    
    # 注册python回调
    call_backs = {
        "call_back" : py_call_back,
        "call_back_dump" : py_call_back_dump,
    }
    for call_back_name, call_back in call_backs.items():
        nic.registerGeneralHandler(call_back_name, call_back)
        
    while (True):
        pass
    
    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

recv_frame()

## 3.3. 使用SDR设备发送WiFi包
PyPicoScenes提供丰富的物理层参数控制能力，开发者可通过API精准配置以下发射链路参数：​​多通道选择​​（setTxChannels指定物理端口索引）、​​射频带宽​​（通过setTxSamplingRate和PicoScenesFrameTxParameters.cbw设置工作带宽）、​​帧格式设置​​（支持VHT/EHTSU等802.11ac/ax/be协议帧格式）、​​保护间隔​​（可以配置GI_800/GI_3200等抗多径参数）、​​信道编码​​（BCC/LDPC编码方式）、​​发射功率​​（setTxpower支持比例系数和绝对值模式）、​​中心频率​​（setCarrierFrequency）及​​天线阵列​​（setTxAntennas选择天线），可满足MIMO预编码验证、协议栈兼容性测试及高密度信号模拟等复杂场景需求。

### 3.3.1. 单设备单通道发射配置
执行以下代码将生成并发送中心频率为5300MHz、40MHz带宽的802.11ac VHT格式帧。

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def txcm2channel(txcm:int = 1):
    tx_channels = []
    for i in range(32):
        if txcm & (1 << i):
            tx_channels.append(i)
    return tx_channels

def transmit_frame(nicName:str = 'usrp'):
    # result = subprocess.run(["PicoScenes", "-q"], capture_output=True, text=True)
    # 以root权限执行当前脚本
    run_as_root()
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取usrp或者网卡设备
    nic = getNic(nicName)
    frontEnd = nic.getTypedFrontEnd[AbstractSDRFrontEnd]()

    # 参数设置
    # 设置txcm为1（tx-channel:[0]）
    txChannelList = [0]
    frontEnd.setTxChannels(txChannelList)
    nic.getUserSpecifiedTxParameters().txcm = frontEnd.getTxChainMask()

    ## 设置时钟源，默认为"internal"
    frontEnd.setClockSource("internal")
    frontEnd.setTimeSource("internal")

    ## 设置发送包的中心频率为5300e6
    frontEnd.setCarrierFrequency(5300e6)

    ## 设置发送功率为最大发送功率的0.75
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(0.75)
    
    ''' ## 设置发送功率为20dBm
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(20)
    '''

    ## 设置agc，默认关闭
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setAGC(False)

    ## 设置发送天线，默认为"TX/RX"
    frontEnd.setTxAntennas(["TX/RX", "TX/RX"])

    ## 设置N-thread multithread Rx decoding，默认为1
    frontEnd.setNumThreads4RxDecoding(1)
    
    ## 设置发送包格式为TX_CBW_40_VHT
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: VHT
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding
    txParameters.coding[0] = ChannelCodingEnum.BCC
    
    ## 设置mcs，默认为0
    txParameters.mcs[0] = 0
    ## 设置sts，默认为1
    txParameters.numSTS[0] = 1
    ## 设置ess，默认为0
    txParameters.numExtraSounding = 0

    ''' ## 设置发送包的格式为TX_CBW_40_VHT_LDPC
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: VHT
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''
    
    ''' ## 设置发送包的格式为TX_CBW_20_EHTSU 
    ### 设置TxSamplingRate为20e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(20e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: EHTSU
    txParameters.frameType = PacketFormatEnum.PacketFormat_EHTSU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    ### 设置cbw: 20
    txParameters.cbw = ChannelBandwidthEnum.CBW_20
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''

    cppyy.gbl.setTxParameters(nic, txParameters)
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("usrp")

`tx-channel`和`txcm`是功能完全相同的两个选项，均用于配置发射通道，只是参数格式不同：`tx-channel`采用直接列举通道编号的方式（如`0,1,2,3`），而`txcm`使用位掩码形式（如`15`对应二进制`1111`表示启用全部4个射频通道）。需要特别注意的是，多通道发射仅代表物理层资源的配置，并不等同于MIMO传输技术。`txcm`编码规则如下：
* 1（二进制为01）：仅第1发送通道，对应channelList为[0]；
* 2（二进制为10）：仅第2发送通道，对应channelList为[1]；
* 3（二进制为11）：第1和第2发送通道，对应channelList为[0,1]；
* 4（二进制100）：第三发送通道，对应channelList为[2]；

### 3.3.2. 多设备多通道发送
​​PicoScenes支持通过单个或多个NI USRP设备组合实现多通道发送。​​

#### 3.3.2.1. ​​基于NI USRP设备的单空间流（1-STS）帧多通道发送
假设您的两台usrp设备的设备ID分别为usrp192.168.30.2和usrp192.168.70.2，执行下面代码可以使用4根天线发送1-STS的WiFi包。

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeWorkingMode = cppyy.gbl.EchoProbeWorkingMode
EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType
EchoProbeReplyStrategy = cppyy.gbl.EchoProbeReplyStrategy
EchoProbeParameters = cppyy.gbl.EchoProbeParameters

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def txcm2channel(txcm:int = 1):
    tx_channels = []
    for i in range(32):
        if txcm & (1 << i):
            tx_channels.append(i)
    return tx_channels

def transmit_frame(nicName:str = 'usrp'):
    # result = subprocess.run(["PicoScenes", "-q"], capture_output=True, text=True)
    # 以root权限执行当前脚本
    run_as_root()
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取usrp或者网卡设备
    nic = getNic(nicName)
    frontEnd = nic.getTypedFrontEnd[AbstractSDRFrontEnd]()

    # 参数设置
    ## 设置txchannel
    txChannelList = [0,1,2,3]
    frontEnd.setTxChannels(txChannelList)
    nic.getUserSpecifiedTxParameters().txcm = frontEnd.getTxChainMask()

    # ## 设置tx-cbw
    # nic.getUserSpecifiedTxParameters().cbw = ChannelBandwidthEnum(20)

    ## 设置时钟源，默认为"external"
    frontEnd.setClockSource("external")
    frontEnd.setTimeSource("external")

    ## 设置发送包的中心频率为5300e6
    frontEnd.setCarrierFrequency(5300e6)

    ## 设置发送功率为最大发送功率的0.75
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(0.75)
    
    ''' ## 设置发送功率为20dBm
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(20)
    '''
    ## 设置agc，默认关闭
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setAGC(False)

    ## 设置发送天线，默认为"TX/RX"
    frontEnd.setTxAntennas(["TX/RX", "TX/RX"])

    ## 设置N-thread multithread Rx decoding，默认为1
    frontEnd.setNumThreads4RxDecoding(1)
    
    ## 设置发送包的格式为TX_CBW_40_EHTSU 
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: EHTSU
    txParameters.frameType = PacketFormatEnum.PacketFormat_EHTSU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC

    ''' ## 设置发送包的格式为TX_CBW_40_VHT_LDPC
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: VHT
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''

    cppyy.gbl.setTxParameters(nic, txParameters)
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("usrp192.168.30.2,192.168.70.2")

#### 3.3.2.2. 基于NI USRP设备的MIMO帧多通道（射频链路）发射​
假设您的两台usrp设备的设备ID分别为usrp192.168.30.2和usrp192.168.70.2，执行下面代码可以使用4根天线发射MIMO帧。

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeWorkingMode = cppyy.gbl.EchoProbeWorkingMode
EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType
EchoProbeReplyStrategy = cppyy.gbl.EchoProbeReplyStrategy
EchoProbeParameters = cppyy.gbl.EchoProbeParameters

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def txcm2channel(txcm:int = 1):
    tx_channels = []
    for i in range(32):
        if txcm & (1 << i):
            tx_channels.append(i)
    return tx_channels

def transmit_frame(nicName:str = 'usrp'):
    # result = subprocess.run(["PicoScenes", "-q"], capture_output=True, text=True)
    # 以root权限执行当前脚本
    run_as_root()
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取usrp或者网卡设备
    nic = getNic(nicName)
    frontEnd = nic.getTypedFrontEnd[AbstractSDRFrontEnd]()

    # 参数设置
    ## 设置txchannel
    txChannelList = [0,1,2,3]
    frontEnd.setTxChannels(txChannelList)
    nic.getUserSpecifiedTxParameters().txcm = frontEnd.getTxChainMask()

    # ## 设置tx-cbw
    # nic.getUserSpecifiedTxParameters().cbw = ChannelBandwidthEnum(20)

    ## 设置时钟源，默认为"external"
    frontEnd.setClockSource("external")
    frontEnd.setTimeSource("external")

    ## 设置发送包的中心频率为5300e6
    frontEnd.setCarrierFrequency(5300e6)

    ## 设置发送功率为最大发送功率的0.75
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(0.75)
    
    ''' ## 设置发送功率为20dBm
    if (frontEnd.getTxChainMask() != 0):
        frontEnd.setTxpower(20)
    '''
    ## 设置agc，默认关闭
    if (frontEnd.getRxChainMask() != 0):
        frontEnd.setAGC(False)

    ## 设置发送天线，默认为"TX/RX"
    frontEnd.setTxAntennas(["TX/RX", "TX/RX"])

    ## 设置N-thread multithread Rx decoding，默认为1
    frontEnd.setNumThreads4RxDecoding(1)
    
    ## 设置发送包的格式为TX_CBW_40_EHTSU 
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: EHTSU
    txParameters.frameType = PacketFormatEnum.PacketFormat_EHTSU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
     ### 设置sts
    txParameters.numSTS[0] = 4
    
    ''' ## 设置发送包的格式为TX_CBW_40_VHT_LDPC
    ### 设置TxSamplingRate为40e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(40e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: VHT
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    ### 设置cbw: 40
    txParameters.cbw = ChannelBandwidthEnum.CBW_40
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''
    
    ''' ## 设置发送包的格式为TX_CBW_20_EHTSU 
    ### 设置TxSamplingRate为20e6
    if (not frontEnd.getHardwareSupportedTxChannels().empty()):
        frontEnd.setTxSamplingRate(20e6)
    ### 设置tx-resample-ratio
    frontEnd.setTxResampleRatio(1)
    txParameters = nic.getUserSpecifiedTxParameters()
    ### 设置format: EHTSU
    txParameters.frameType = PacketFormatEnum.PacketFormat_EHTSU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    ### 设置cbw: 20
    txParameters.cbw = ChannelBandwidthEnum.CBW_20
    ### 设置coding为LDPC
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''
    cppyy.gbl.setTxParameters(nic, txParameters)
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("usrp192.168.30.2,192.168.70.2")

## 3.4. 使用AX210/AX200网卡接收WiFi包并测量CSI
为了使用网卡接收WiFi包，首先需要在命令行设置网卡的信道和带宽，具体步骤如下：
* 首先需要使用**array_status**查看网卡的<PHYPath_ID>，假设是4；
* 然后执行**array_prepare_for_picoscenes** <PHYPath_ID> <CHANNEL_CONFIG>命令设置网卡，例如
```bash
    array_prepare_for_picoscenes 4 "2412 HT20"
```
其中<PHYPath_ID>是网卡的ID，<CHANNEL_CONFIG>是WiFi信道配置。关于信道配置更详细的信息请参考[这里](https://ps.zpj.io/channels.html)。


### 3.4.1. 监听Wi-Fi流量并测量802.11a/g/n/ac/ax/be格式帧的CSI
在使用网卡接收WiFi包之前，需要使用array_prepare_for_picoscenes命令设定网卡信道和带宽。例如：
```bash
# 监听"2422 HT20"信道
array_prepare_for_picoscenes 4 "2422 HT20"
# 监听"5240 HT20"信道
array_prepare_for_picoscenes 4 "5240 HT20"
# 监听"5200 80 5210"信道
array_prepare_for_picoscenes 4 "5200 80 5210"
```
然后执行如下代码接收WiFi包并测量CSI：

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess

FrameDumper = cppyy.gbl.FrameDumper

# 简单的回调函数
def py_call_back(frame):
    print("-----------------------------get one frame----------------------------")
    return True

# python回调接收到frame并将frame保存到文件
def py_call_back_dump(frame):
    fileName = "testCSI"
    print(f"dump a frame to {fileName}")
    # 保存frame到文件
    FrameDumper.getInstanceWithoutTime(fileName).dumpRxFrame(frame)
    return True

def recv_frame(nicName:str = '4'):
    # 以root权限执行当前脚本
    run_as_root()
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取网卡
    nic = getNic(nicName)
    
    # 启动网卡接收服务
    nic.startRxService()

    # 注册python回调
    call_backs = {
        "call_back" : py_call_back,
        "call_back_dump" : py_call_back_dump,
    }
    for call_back_name, call_back in call_backs.items():
        nic.registerGeneralHandler(call_back_name, call_back)
    while (True):
        pass

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

recv_frame("4")

## 3.5. 使用AX210/AX200网卡发送WiFi包
为了使用网卡发送WiFi包，首先需要在命令行设置网卡的信道和带宽，具体步骤如下：
* 首先需要使用**array_status**查看网卡的<PHYPath_ID>，假设是4；
* 然后执行**array_prepare_for_picoscenes** <PHYPath_ID> <CHANNEL_CONFIG>命令设置网卡，例如
```bash
    array_prepare_for_picoscenes 4 "2412 HT20"
```
其中<PHYPath_ID>是网卡的ID，<CHANNEL_CONFIG>是WiFi信道配置。关于信道配置更详细的信息请参考[这里](https://ps.zpj.io/channels.html)。


### 3.5.1. 发送802.11a/g/n/ac/ax/be格式的WiFi包
设置完网卡之后，执行下面代码发送格式为TX_CBW_20_HT的WiFi包。

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeWorkingMode = cppyy.gbl.EchoProbeWorkingMode
EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType
EchoProbeReplyStrategy = cppyy.gbl.EchoProbeReplyStrategy
EchoProbeParameters = cppyy.gbl.EchoProbeParameters

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def transmit_frame(nicName:str = '4'):
    # 以root权限运行当前脚本
    run_as_root()
    # 在脚本里面设置网卡信道
    result = subprocess.run(["array_prepare_for_picoscenes", nicName ,"2412 HT20"], capture_output=True, text=True)
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取网卡
    nic = getNic(nicName)

    ## 发送TX_CBW_20_HT格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_HT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    txParameters.cbw = ChannelBandwidthEnum.CBW_20
    txParameters.coding[0] = ChannelCodingEnum.BCC
    
    '''
    ## 发送TX_CBW_160_HESU格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_HESU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    '''
    
    ''' ## 发送TX_CBW_160_VHT格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.BCC
    '''
    
    cppyy.gbl.setTxParameters(nic, txParameters)
    
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))
        
    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("4")


### 3.5.2. MCS配置与天线选择
PyPicoScenes支持为AX210/AX200网卡指定调制编码方案（MCS）参数及收发天线选择。下面说明如何使用天线2发送MCS为5格式为TX_CBW_160_HESU的WiFi包。首先在命令行设置网卡的信道和带宽：
```bash
array_prepare_for_picoscenes 4 "5640 160 5250"
```
然后执行以下代码：

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeWorkingMode = cppyy.gbl.EchoProbeWorkingMode
EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType
EchoProbeReplyStrategy = cppyy.gbl.EchoProbeReplyStrategy
EchoProbeParameters = cppyy.gbl.EchoProbeParameters

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def transmit_frame(nicName:str = '4'):
    # 以root权限运行当前脚本
    run_as_root()
    # result = subprocess.run(["array_prepare_for_picoscenes", nicName ,"5640 160 5250"], capture_output=True, text=True)
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取网卡
    nic = getNic(nicName)

    ## 设置txcm为2
    nic.getTypedFrontEnd[MAC80211CSIExtractableFrontEnd]().setTxChainMask(2)
    
    ## 发送TX_CBW_160_HESU格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_HESU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    
    ## 设置mcs为5
    txParameters.mcs[0] = 5
    
    ''' ## 发送TX_CBW_160_VHT格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.BCC
    '''
    cppyy.gbl.setTxParameters(nic, txParameters)
    
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("4")


### 3.5.3. 实时指定网卡的信道与带宽
PyPicoScenes提供了API用于实时指定网卡的信道与带宽，而不必重新执行array_prepare_for_picoscenes命令。如果您的AX210/AX200网卡的<PHYPath_ID>是4，并且当前工作在"5180 80 5210"信道，执行以下的代码，可以让其直接工作在"5955 160 6025"信道。

In [ ]:
from PyPicoScenes import *
from PyPicoScenes import run_as_root
import subprocess
import os
import sys
import random
import getpass

cppyy.include("PyPicoScenesCommon.hpp")
FrameDumper = cppyy.gbl.FrameDumper

EchoProbeWorkingMode = cppyy.gbl.EchoProbeWorkingMode
EchoProbeInjectionContent = cppyy.gbl.EchoProbeInjectionContent
EchoProbePacketFrameType = cppyy.gbl.EchoProbePacketFrameType
EchoProbeReplyStrategy = cppyy.gbl.EchoProbeReplyStrategy
EchoProbeParameters = cppyy.gbl.EchoProbeParameters

def buildBasicFrame(taskId, frameType, nic, **kwargs):
    frame = nic.initializeTxFrame()
    if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.NDP:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters()).txParameters.NDPFrame = True
    else:
        frame.setTxParameters(nic.getUserSpecifiedTxParameters())
        frame.setTaskId(taskId)
        frame.setPicoScenesFrameType(frameType)
        if frameType == EchoProbePacketFrameType.SimpleInjectionFrameType and kwargs.get("injectorContent") == EchoProbeInjectionContent.Full:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if frameType == EchoProbePacketFrameType.EchoProbeRequestFrameType:
            frame.addSegment(std.make_shared[ExtraInfoSegment](nic.getFrontEnd().buildExtraInfo()))
        if kwargs.get("randomPayloadLength"):
            l = int(kwargs["randomPayloadLength"])
            vec = [random.randint(0,255) for _ in range(l)]
            segment = std.make_shared[PayloadSegment]("RandomPayload", vec, PayloadDataType.RawData)
            frame.addSegment(segment)

    sourceAddr = nic.getFrontEnd().getMacAddressPhy()
    if kwargs.get("randomMAC"):
        sourceAddr[0] = random.randint(0,255)
        sourceAddr[1] = random.randint(0,255)
    frame.setSourceAddress(sourceAddr.data())
    frame.setDestinationAddress(MagicIntel123456.data())
    frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
    frame.txParameters.forceSounding = True

    if kwargs.get("inj_for_intel5300"):
        frame.setSourceAddress(MagicIntel123456.data())
        frame.setDestinationAddress(MagicIntel123456.data())
        frame.set3rdAddress(nic.getFrontEnd().getMacAddressPhy().data())
        frame.setForceSounding(False)
        frame.setChannelCoding(ChannelCodingEnum.BCC)
    return frame

def transmit_frame(nicName:str = '4'):
    # 以root权限执行当前脚本
    run_as_root()
    # result = subprocess.run(["array_prepare_for_picoscenes", nicName ,"5640 160 5250"], capture_output=True, text=True)
    # 启动PicoScenes平台
    picoscenes_start()
    # 获取网卡
    nic = getNic(nicName)

    ## 重新设定网卡的信道
    control = 5180e6
    rxcbw = 80e6
    freq = 5210e6
    nic.getFrontEnd().setChannelAndBandwidth(control, rxcbw, freq)
    
    ## 设置txcm为2
    nic.getTypedFrontEnd[MAC80211CSIExtractableFrontEnd]().setTxChainMask(2)
    
    ## 发送TX_CBW_160_HESU格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_HESU
    txParameters.guardInterval = GuardIntervalEnum.GI_3200
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.LDPC
    
    ''' ## 发送TX_CBW_160_VHT格式的包
    txParameters = nic.getUserSpecifiedTxParameters()
    txParameters.frameType = PacketFormatEnum.PacketFormat_VHT
    txParameters.guardInterval = GuardIntervalEnum.GI_800
    txParameters.cbw = ChannelBandwidthEnum.CBW_160
    txParameters.coding[0] = ChannelCodingEnum.BCC
    '''
    
    ## 设置mcs为5
    txParameters.mcs[0] = 5
    cppyy.gbl.setTxParameters(nic, txParameters)
    
    # 启动网卡服务
    nic.startTxService()

    cf_repeat = 1000
    tx_delay_us = 5e3

    for i in range(int(cf_repeat)):
        taskId = SystemTools.Math.uniformRandomNumberWithinRange[std.uint16_t](9999, 65535)
        txframe = buildBasicFrame(taskId, EchoProbePacketFrameType.SimpleInjectionFrameType, nic)
        nic.transmitPicoScenesFrameSync(txframe)
        SystemTools.Time.delay_periodic(int(tx_delay_us))

    # 停止网卡的接收服务
    nic.stopRxService()
    # 停止网卡的发送服务
    nic.stopTxService()
    # 停止PicoScenes平台
    picoscenes_stop()
    # 在调用picoscenes_stop()之前，picoscenes_wait()会一直阻塞
    picoscenes_wait()

transmit_frame("4")


## 3.5. 注意事项
​​3.5. 注意事项​​
PyPicoScenes 通过 cppyy 的动态绑定技术，实现了对 PicoScenes C++ API 的高效封装。开发者只需在 Python 脚本中导入对应的头文件（include("PicoScenes/SystemTools.hxx")）和动态库（load_library("libFrontEnd")），即可直接调用底层 API 实现无线信号收发、CSI 文件解析等核心功能。PyPicoScenes 的 API 接口与原生 C++ 版本完全一致，具体使用方法可参考 PicoScenes 原生 API 文档。得益于 cppyy 的实时解析机制，Python 层可直接操作硬件控制逻辑（如设置 USRP 采样率、配置 WiFi 信道参数等），且行为与 C++ 实现严格一致，开发者需注意在跨平台部署时验证动态库路径和环境依赖的兼容性。